<pre>
Persistence in LangGraph refers to the ability to save and restore thestate of a workflow over time.
There are two main components of persistence in LangGraph:
1. Checkpointer : it is used to save the current state of the workflow at specific points in time and also store all intermediate
state value. 
2. Thread : it is used thread_id to uniquely identify and manage different workflow executions.

</pre>

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()

True

In [12]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

In [13]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [14]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt).content
    return {'joke': response}
  
def generate_explanation(state: JokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.invoke(prompt).content
    return {'explanation': response}

In [15]:
graph = StateGraph(JokeState)

graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explanation", generate_explanation)

graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "generate_explanation")
graph.add_edge("generate_explanation", END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [16]:
config1 = {"configurable":{"thread_id":"1"}}
workflow.invoke({"topic":"technology"}, config=config1)

{'topic': 'technology',
 'joke': 'Why did the computer get glasses?\n\nBecause it needed to improve its **web-site**!',
 'explanation': 'Here\'s the explanation for the joke:\n\nThis joke is a **pun**, which means it plays on words that sound alike but have different meanings.\n\n*   **"Web-site"** sounds very similar to **"website"**.\n*   A **website** is a collection of pages on the internet that a computer displays.\n*   **"Web-site"** can also be interpreted as the computer\'s **"web"** (referring to its connection to the internet or its internal network) and its **"site"** (meaning its location or how it\'s perceived).\n\nSo, the joke works because:\n\n1.  **Literal interpretation:** Computers need to "see" or display websites clearly. If a computer\'s "vision" for websites is poor, it might need glasses to improve its ability to view them.\n2.  **Pun interpretation:** The phrase "improve its web-site" is a clever way of saying the computer needed to improve its **website** (the 

In [19]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'technology', 'joke': 'Why did the computer get glasses?\n\nBecause it needed to improve its **web-site**!', 'explanation': 'Here\'s the explanation for the joke:\n\nThis joke is a **pun**, which means it plays on words that sound alike but have different meanings.\n\n*   **"Web-site"** sounds very similar to **"website"**.\n*   A **website** is a collection of pages on the internet that a computer displays.\n*   **"Web-site"** can also be interpreted as the computer\'s **"web"** (referring to its connection to the internet or its internal network) and its **"site"** (meaning its location or how it\'s perceived).\n\nSo, the joke works because:\n\n1.  **Literal interpretation:** Computers need to "see" or display websites clearly. If a computer\'s "vision" for websites is poor, it might need glasses to improve its ability to view them.\n2.  **Pun interpretation:** The phrase "improve its web-site" is a clever way of saying the computer needed to improve it

In [20]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'technology', 'joke': 'Why did the computer get glasses?\n\nBecause it needed to improve its **web-site**!', 'explanation': 'Here\'s the explanation for the joke:\n\nThis joke is a **pun**, which means it plays on words that sound alike but have different meanings.\n\n*   **"Web-site"** sounds very similar to **"website"**.\n*   A **website** is a collection of pages on the internet that a computer displays.\n*   **"Web-site"** can also be interpreted as the computer\'s **"web"** (referring to its connection to the internet or its internal network) and its **"site"** (meaning its location or how it\'s perceived).\n\nSo, the joke works because:\n\n1.  **Literal interpretation:** Computers need to "see" or display websites clearly. If a computer\'s "vision" for websites is poor, it might need glasses to improve its ability to view them.\n2.  **Pun interpretation:** The phrase "improve its web-site" is a clever way of saying the computer needed to improve i

<code>Time Travel Debugging</code>

In [21]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0f1011-a42e-6fa6-8000-15c21cc3c45a"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0f1011-a42e-6fa6-8000-15c21cc3c45a'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [23]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0f101e-9af7-63e8-8002-9b9acf53f48d"}})

{'topic': 'technology',
 'joke': 'Why did the computer get glasses?\n\nBecause it needed to improve its **web-site**!',
 'explanation': 'Here\'s the explanation for the joke:\n\nThis joke is a **pun**, which means it plays on words that sound alike but have different meanings.\n\n*   **"Web-site"** sounds very similar to **"website"**.\n*   A **website** is a collection of pages on the internet that a computer displays.\n*   **"Web-site"** can also be interpreted as the computer\'s **"web"** (referring to its connection to the internet or its internal network) and its **"site"** (meaning its location or how it\'s perceived).\n\nSo, the joke works because:\n\n1.  **Literal interpretation:** Computers need to "see" or display websites clearly. If a computer\'s "vision" for websites is poor, it might need glasses to improve its ability to view them.\n2.  **Pun interpretation:** The phrase "improve its web-site" is a clever way of saying the computer needed to improve its **website** (the 

In [24]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'technology', 'joke': 'Why did the computer get glasses?\n\nBecause it needed to improve its **web-site**!', 'explanation': 'Here\'s the explanation for the joke:\n\nThis joke is a **pun**, which means it plays on words that sound alike but have different meanings.\n\n*   **"Web-site"** sounds very similar to **"website"**.\n*   A **website** is a collection of pages on the internet that a computer displays.\n*   **"Web-site"** can also be interpreted as the computer\'s **"web"** (referring to its connection to the internet or its internal network) and its **"site"** (meaning its location or how it\'s perceived).\n\nSo, the joke works because:\n\n1.  **Literal interpretation:** Computers need to "see" or display websites clearly. If a computer\'s "vision" for websites is poor, it might need glasses to improve its ability to view them.\n2.  **Pun interpretation:** The phrase "improve its web-site" is a clever way of saying the computer needed to improve i

<code>Updating State</code>

In [28]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0f101e-7fe3-6e7d-bfff-55ad82555e13", "checkpoint_ns": ""}},{"topic":"science"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0f1029-1ad4-66df-8000-d3baf890c2ec'}}

In [29]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'science'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1029-1ad4-66df-8000-d3baf890c2ec'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2026-01-14T04:36:18.476617+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f101e-7fe3-6e7d-bfff-55ad82555e13'}}, tasks=(PregelTask(id='e325abf7-f25a-0e5c-edb8-0b76f9303081', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'technology', 'joke': 'Why did the computer get glasses?\n\nBecause it needed to improve its **web-site**!', 'explanation': 'Here\'s the explanation for the joke:\n\nThis joke is a **pun**, which means it plays on words that sound alike but have different meanings.\n\n*   **"Web-site"** sounds very similar to **"website"**.\n*   A **website** is

In [36]:
# None means to continue from the last state
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0f1029-1ad4-66df-8000-d3baf890c2ec"}})

{'topic': 'science',
 'joke': 'Why did the atom break up with the molecule?\n\nBecause he felt like he was being *bonded* too much!',
 'explanation': 'This joke is a play on words, using the scientific term "bond" in a way that also has a common, non-scientific meaning. Here\'s the breakdown:\n\n*   **Scientific Context:** In chemistry, atoms join together to form molecules through **bonds**. These bonds are forces that hold atoms together. So, in a scientific sense, atoms are "bonded" to molecules.\n\n*   **Everyday Meaning of "Bonded":** In everyday language, "bonded" can mean being very close, attached, or even feeling suffocated or overly controlled by someone. If someone feels "bonded too much," it implies they feel like they have too many obligations, too much pressure, or are not getting enough personal space.\n\n*   **The Punchline:** The joke uses this double meaning. The atom is "breaking up" with the molecule, and the reason given is that he felt "bonded too much."\n    *   

In [37]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'science', 'joke': 'Why did the atom break up with the molecule?\n\nBecause he felt like he was being *bonded* too much!', 'explanation': 'This joke is a play on words, using the scientific term "bond" in a way that also has a common, non-scientific meaning. Here\'s the breakdown:\n\n*   **Scientific Context:** In chemistry, atoms join together to form molecules through **bonds**. These bonds are forces that hold atoms together. So, in a scientific sense, atoms are "bonded" to molecules.\n\n*   **Everyday Meaning of "Bonded":** In everyday language, "bonded" can mean being very close, attached, or even feeling suffocated or overly controlled by someone. If someone feels "bonded too much," it implies they feel like they have too many obligations, too much pressure, or are not getting enough personal space.\n\n*   **The Punchline:** The joke uses this double meaning. The atom is "breaking up" with the molecule, and the reason given is that he felt "bonded 